In [1]:
# import packages

import pandas as pd
import numpy as np
import json

# Read csv files

user = pd.read_csv('user.csv', header=0, na_filter=False)
business = pd.read_csv('business.csv', header=0, na_filter=False)
review = pd.read_csv('review.csv', header=0, na_filter=False)

In [18]:
# lat / long binning

business_x = business.latitude
business_y = business.longitude

step = 1

to_bin = lambda x: np.floor(x / step) * step

business["latbin"] = business_x.map(to_bin)
business["lonbin"] = business_y.map(to_bin)


# SQL filtering and joins

business_filtered = business.loc[(business.lonbin == -81) & (business.latbin == 35)];
review_filt = pd.merge(review, business_filtered, left_on='business_id', right_on='id', how='inner');
user_filtered = pd.merge(user, review_filt, left_on='id', right_on='user_id', how='inner');
user_filt = user_filtered.drop_duplicates('id', keep='first');
business_filtered = pd.merge(user_filt, business_filtered, left_on='business_id', right_on='id', how='inner');
business_filt = business_filtered.drop_duplicates('business_id', keep='first');

# Joining from businesses to reviews to users will provide data on users who have commented on businesses from
# a specified location. Further merging in last line is to retrieve additional businesses not located in same geo area.

In [24]:
business_filtered.T.shape

(57, 77986)

In [26]:
business_filt.T

,0,2,240,244,251,427,430,474,479,492,...,77976,77977,77978,77979,77980,77981,77982,77983,77984,77985
id_x,--0RtXvcOIE4XbErYca6Rw,--3WaS23LcIXtxyFULJHTA,--44NNdtngXMzsxyN7ju6Q,--6D_IuxyKTN53pHi904ag,--6Ke7_lBBM6XAramtPoWw,--7D3lFxyMYvs2JYiRrg6Q,--cBKDfZ4AbM9U90VkFyBg,--i7WS2pwOQKQl6uB15q1A,--JS-RvSykutl1DavCbkIg,--LnqQQ2mVdrpPt2UC8d6A,...,_RdZw-nsuyH4PHIRUXIQow,_somx1aNJ2F1Du_0FYJRVA,_SPTgvMEM-278HFOW1OZVg,_t5X0o9UeIrz1Wy_fMutLw,_T8B2prKgegfasOQ06Wbuw,_V4rTnQt-vev1HngbjkhIQ,_VDJ23dt2wbtXfbofOST-A,_veMggEvnMCGcX4Wet36DQ,_xgjhgNZ5H6qQolujfDeRQ,_Y8qkFCesjTp2vN3c98t6w
name_x,Ryan,Kristie,Austin,Mary,Laura,Bill,Monica,Pamela,Mary,Kenneth,...,H,Nakia,Devona,Jim,Terri,Peter,Richard,Jay,Lee,J
review_count_x,2,213,2,2,15,2,9,2,2,1,...,3,1,4,1,5,3,3,9,372,1
yelping_since,2013-05-30 00:00:00,2010-05-02 00:00:00,2013-01-22 00:00:00,2013-02-06 00:00:00,2009-09-12 00:00:00,2016-04-29 00:00:00,2012-12-26 00:00:00,2017-05-06 00:00:00,2014-07-07 00:00:00,2012-04-08 00:00:00,...,2012-10-16 00:00:00,2016-01-17 00:00:00,2014-03-28 00:00:00,2016-02-11 00:00:00,2017-04-11 00:00:00,2012-05-24 00:00:00,2010-06-16 00:00:00,2014-04-09 00:00:00,2007-08-28 00:00:00,2013-08-12 00:00:00
useful_x,0,63,0,0,19,0,1,0,1,0,...,0,0,0,0,0,0,0,0,2700,0
funny_x,0,6,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1143,0
cool_x,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1175,0
fans,0,10,0,0,6,0,0,0,1,0,...,0,0,0,0,0,0,0,0,38,0
average_stars,4.5,3.93,5,5,4.41,5,4.56,3.5,1,2,...,2.33,5,2.6,3,3.8,2.33,2.33,2.78,3.07,5
compliment_hot,0,11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,29,0


In [12]:
# Select from csv to json (Users)

userpoints = pd.DataFrame([user.id, user.name, user.review_count, user.average_stars]).T; # select rows

userpoints2 = userpoints.to_json(orient='records'); # convert to json

with open('users.json', 'w') as f: # save to file
    f.write(userpoints2)

In [35]:
# Select from csv to json (Businesses)

businesspoints = pd.DataFrame([business.id, business.name, business.review_count, business.stars]).T; # select rows

businesspoints2 = businesspoints.to_json(orient='records'); # convert to json

with open('business.json', 'w') as f: # save to file
    f.write(businesspoints2)

In [11]:
# Select from csv to json (Reviews)

# x = review.text.str[:50]; // to truncate comments to 50 characters

reviews = pd.DataFrame([review.business_id, review.user_id]).T; # select rows

reviews2 = reviews.to_json(orient='records'); # convert to json

with open('review.json', 'w') as f:  # save to file
    f.write(reviews2)

In [11]:
user.T.info

<bound method DataFrame.info of                                    0                       1        \
id                  ---1lKK3aKOuomHnwAkAow  ---94vtJ_5o_nikEs6hUjg   
name                                Monera                     Joe   
review_count                           245                       2   
yelping_since          2007-06-04 00:00:00     2016-05-27 00:00:00   
useful                                  67                       0   
funny                                   22                       0   
cool                                     9                       0   
fans                                    15                       0   
average_stars                         3.97                       5   
compliment_hot                           2                       0   
compliment_more                          3                       0   
compliment_profile                       2                       0   
compliment_cute                          1                